In [45]:
import random
import numpy as np
import pandas as pd
# Algoritmos Genéticos
import yfinance as yf
from deap import algorithms, base, creator, tools

In [46]:
df_original = yf.Ticker('NQ=F')
df_original = df_original.history(start='2010-01-01', end='2024-01-01')

### Función de backtest para cada combinación de parámetros


In [47]:
def backtest_strategy(sma_1, sma_2):
    # Asegurarse de que sean enteros
    sma_1 = int(sma_1)
    sma_2 = int(sma_2)
    
    # Hacer una copia del DataFrame original para cada evaluación
    df = df_original.copy()
    
    # Calcular medias móviles
    df['SMA_50'] = df['Close'].rolling(window=sma_1).mean()
    df['SMA_200'] = df['Close'].rolling(window=sma_2).mean()
    
    # Definir reglas de entrada y salida
    df['Signal'] = 0
    df.loc[df['SMA_50'] > df['SMA_200'], 'Signal'] = 1
    df.loc[df['SMA_50'] < df['SMA_200'], 'Signal'] = -1
    
    # Simular la ejecución de las órdenes
    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    
    # Eliminar NaNs
    df.dropna(inplace=True)
    
    # Rendimiento total de la estrategia
    return (df['Strategy_Returns'] + 1).prod() - 1

### Función de evaluación


In [48]:
def evaluate(individual):
    sma_1, sma_2 = individual
    # Validar que ambos valores sean mayores que 0 y que SMA_1 < SMA_2
    if sma_1 <= 0 or sma_2 <= 0 or sma_1 >= sma_2:
        return -np.inf,  # Penalización por una configuración inválida
    total_return = backtest_strategy(sma_1, sma_2)
    return (total_return,)

### Crear individuo y configuración genética


In [49]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()


/home/antonio/anaconda3/envs/data_science_path/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/antonio/anaconda3/envs/data_science_path/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


### Registro de los atributos de las medias móviles como enteros


In [50]:
toolbox.register("attr_sma1", random.randint, 10, 100)
toolbox.register("attr_sma2", random.randint, 100, 300)
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (toolbox.attr_sma1, toolbox.attr_sma2), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

### Registro de la función de evaluación


In [51]:
toolbox.register("evaluate", evaluate)

### Operadores genéticos: cruza y mutación ajustados para enteros


In [53]:
toolbox.register("mate", tools.cxUniform, indpb=0.5)  # cxUniform mezcla valores enteros
toolbox.register("mutate", tools.mutUniformInt, low=[10, 100], up=[100, 300], indpb=0.2)  # mutUniformInt para mantener enteros
toolbox.register("select", tools.selTournament, tournsize=3)

### Ejecutar algoritmo genético

In [54]:
population = toolbox.population(n=10)

### Algoritmo evolutivo simple

In [55]:
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=10, verbose=True)

gen	nevals
0  	10    
1  	8     
2  	10    
3  	8     
4  	9     
5  	8     
6  	7     
7  	8     
8  	8     
9  	8     
10 	10    


([[72, 182],
  [72, 271],
  [72, 271],
  [72, 271],
  [72, 271],
  [72, 271],
  [72, 271],
  [72, 271],
  [72, 271],
  [72, 271]],
 [{'gen': 0, 'nevals': 10},
  {'gen': 1, 'nevals': 8},
  {'gen': 2, 'nevals': 10},
  {'gen': 3, 'nevals': 8},
  {'gen': 4, 'nevals': 9},
  {'gen': 5, 'nevals': 8},
  {'gen': 6, 'nevals': 7},
  {'gen': 7, 'nevals': 8},
  {'gen': 8, 'nevals': 8},
  {'gen': 9, 'nevals': 8},
  {'gen': 10, 'nevals': 10}])

### Imprimir los mejores resultados

In [57]:
best_individual = tools.selBest(population, k=1)[0]
best_return = evaluate(best_individual)[0]  # Calcular el mejor rendimiento

In [58]:
print(f'Los mejores parámetros son SMA_1: {int(best_individual[0])}, SMA_2: {int(best_individual[1])}')
print(f'El mejor rendimiento total es: {best_return:.2%}')

Los mejores parámetros son SMA_1: 72, SMA_2: 271
El mejor rendimiento total es: 388.59%
